# Vanilla Braitenberg Experiments

Author: K. Voudouris, 2023 (c) All Rights Reserved.

Contact: kv301@cam.ac.uk; k.voudouris14@googlemail.com; [Twitter @KozzyVoudouris](https://twitter.com/KozzyVoudouris); [GitHub @kozzy97](https://github.com/kozzy97)

Date: July 2023

This script runs a series of hand-coded agents on the object permanence tests and stores the results in a MySQL database. It relies on a few things:

1. All the dependencies are installed, particularly that animalai is installed properly. I recommend using a conda environment and setting up an ipykernel for running this notebook.
2. AnimalAI is installed as an executable in the `env` folder.
3. A recent installation of MySQL, configured with a database, user, and password, as well as the local (or remote) address to store on. MySQL WorkBench is a good IDE for interacting with MySQL (this was created with WorkBench 8.0)
4. A CSV file in the same directory as this notebook called `databaseConnectionDetails.csv`, containing columns `database_name`, `hostname`, `username`, and `password` for database connection, with the values in the next row. This is gitignored.

In [ ]:
import random
import os
import time

from animalai.envs.environment import AnimalAIEnvironment
from animalai.envs.actions import AAIActions, AAIAction

import sys
sys.path.append('../src')

from vanillaBraitenbergAgent import vanillaBraitenberg #import the vanilla braitenberg class
from yamlHandling import find_yaml_files #this function finds the yaml files in a directory.
from yamlHandling import yaml_combinor #this function combines a batch of yaml files and saves the output in a temporary folder. This means we can run inference on batches of tests at once.
from mysqlConnection import databaseConnector #this function permits connection to a mysql database using a CSV file containing details of the db connection.
from mysqlConnection import agentToDB #this function takes a dictionary and ingresses it into a table
from mysqlConnection import removePreviouslyRunInstances #this function takes a set of yaml files and task names and removes any that have already got results in the database.
from mysqlConnection import selectID #this function finds the integer ID for a table given a particular column name and value

## Database Connection

A function for connecting to the database.

In [ ]:
mycursor, connection = databaseConnector('databaseConnectionDetails.csv')

mycursor.close()

print("Connection checked and closed.")

## Paths

Provide the paths to the directory containing the configs being tested over, as well as the path the animal ai environment. Finally, provide a location for generating temporary files of combined configs. This defaults to the parent directory of the github repository, to prevent results being pushed accidentally.

In [ ]:
configuration_folder = "../../configs/tests_agents"

env_path = "../../env/AnimalAI"

temp_folder_location = "../../.."

## Add All Tasks In Directory To Database

Iterate through the directory and find yaml files and their task names.

In [ ]:
rerunInstanceTable = False

mycursor, connection = databaseConnector('databaseConnectionDetails.csv')

yaml_files, task_names = find_yaml_files(configuration_folder)

if rerunInstanceTable:
    dropTable = "DROP TABLE IF EXISTS vbraitenbergvehicleinstanceresults, vbraitenbergvehicleintrainstanceresults, instances;"
    mycursor.execute(dropTable)
    
    sql = "CREATE TABLE instances(instanceid INT AUTO_INCREMENT PRIMARY KEY, instancename VARCHAR(750) UNIQUE NOT NULL);"
    mycursor.execute(sql)

for instance in task_names:
    try:
        insertQuery = "INSERT INTO instances(instancename) VALUES('" + str(instance) + "');"
        mycursor.execute(insertQuery)
        connection.commit()
    except:
        print(f"Task {instance} has already been added to this table. Moving to next.")

mycursor.close()



## Agent

Dictionaries of parameters to define a vanilla braitenberg agent. This agent can interact with the animal-ai environment in a purposeful and meaningful way. The agent's default action is `FORWARDS`. If the agent is currently stationary, then it chooses to go `FORWARDSLEFT` or `FORWARDSRIGHT` with a probability of 0.5. It repeats that action until it is no longer stationary, or if one of the following conditions occurs. If it detects a `GoodGoal`, `GoodGoalBounce`, `GoodGoalMulti`, or a `GoodGoalMultiBounce` directly ahead in its field of view, it chooses the `FORWARDS` action. If it sees one of those objects to its left or right, it turns appropriately until it is directly ahead. If it detects a `DeathZone` or a `BadGoal` directly ahead in its field of view, it chooses the `BACKWARDS` action. If it sees one of those objects to its left or right, it turns appropriately until it is directly ahead. If it sees an immovable object (a `Wall`, `Ramp`, `Tunnel`, etc.) directly ahead, then it chooses to go `FORWARDSLEFT` or `FORWARDSRIGHT` with a probability of 0.5. If the obstacle is still directly ahead, and the agent is not stationary, then it perseverates that action. If the obstacle is to it's left but not directly ahead, it chooses `FORWARDSLEFT`, and vice versa for `right`. This means that the agent can rudimentarily navigate around obstacles.

**Note**
The agent uses raycasting to implement these rules, as well as knowledge of its velocity. Raycasts only apply on the x-z plane of the environment, meaning that the agent cannot detect objects above or below its current y coordinate.

This agent is hand-coded to be able to navigate towards appetitive stimuli and away from aversive stimuli, and to navigate around neutral obstacles. These are general capabilities required of any agent in the Animal-AI Environment. This agent certainly lacks object permanence, as well as many other capabilities that are useful for solving these specific tests of object permanence. However, this agent serves as a good baseline for how we might expect a minimally capable agent to perform on this battery of tests

This agent emits 15 rays spread evenly over the front 60 degrees of the agent's viewing space.

In [ ]:
VBB = {'no_rays' : 15,
       'angles' : 60,
       'aai_seed' : 2023,
       'agent_tag' : 'Vanilla Braitenberg 15 rays over 60 degs'}

In [ ]:
rerunAgentTable = False

mycursor, connection = databaseConnector('databaseConnectionDetails.csv')

if rerunAgentTable:
    dropTable = "DROP TABLE IF EXISTS vbraitenbergvehicleinstanceresults, vbraitenbergvehicleintrainstanceresults, vbraitenbergvehicles;"
    mycursor.execute(dropTable)
    
    sql = "CREATE TABLE `vbraitenbergvehicles` (`agentid` INT AUTO_INCREMENT PRIMARY KEY, `agent_tag` VARCHAR(300), `aai_seed` INT, `no_rays` INT,  `angles` INT, UNIQUE(agent_tag, aai_seed));"
    mycursor.execute(sql)

mycursor.close()

In [ ]:
agent_dict_list = [VBB]

seeds_to_run = [2023, 1997, 356, 1815, 3761] #5 seeds corresponding to eventful years.

Add the agents to the agent table:

In [ ]:
mycursor, connection = databaseConnector('databaseConnectionDetails.csv')

for agent in agent_dict_list:
    for seed in seeds_to_run:
        agent['aai_seed'] = seed
        agentToDB(mycursor, agent, table_name = "vbraitenbergvehicles")

connection.commit()

mycursor.close()

## Run Inference And Store

Need to iterate through the dictionaries and run inference.

In [ ]:
mycursor, connection = databaseConnector('databaseConnectionDetails.csv')

rebuildInstanceResultsTables = False

if rebuildInstanceResultsTables:
    print("Rebuilding results tables, dropping if they already exist.")

    dropInstanceResultsTables = "DROP TABLE IF EXISTS vbraitenbergvehicleinstanceresults, vbraitenbergvehicleintrainstanceresults;"
    mycursor.execute(dropInstanceResultsTables)
    
    createInstanceTable = "CREATE TABLE vbraitenbergvehicleinstanceresults(instanceid INT NOT NULL, agentid INT NOT NULL, finalreward FLOAT(53), FOREIGN KEY (instanceid) REFERENCES instances(instanceid), FOREIGN KEY(agentid) REFERENCES vbraitenbergvehicles(agentid), PRIMARY KEY (instanceid, agentid));"
    mycursor.execute(createInstanceTable)

    createIntraInstanceTable = "CREATE TABLE vbraitenbergvehicleintrainstanceresults(instanceid INT NOT NULL, agentid INT NOT NULL, step INT NOT NULL, actiontaken VARCHAR(30) NOT NULL, stepreward FLOAT(53), xvelocity FLOAT(32), yvelocity FLOAT(32), zvelocity FLOAT(32), xpos FLOAT(32), ypos FLOAT(32), zpos FLOAT(32), FOREIGN KEY (instanceid) REFERENCES instances(instanceid), FOREIGN KEY(agentid) REFERENCES vbraitenbergvehicles(agentid), PRIMARY KEY(instanceid, agentid, step));"
    mycursor.execute(createIntraInstanceTable)

    print("Tables `vbraitenbergvehicleinstanceresults` and `vbraitenbergvehicleintrainstanceresults` have been successfully built.")

mycursor.close()


Define a function to run the experiments. This takes an agent dictionary and first checks whether any results have been recorded for it. If not, then it proceeds with testing. It does testing in batches, generating a temporary yml file to run training on and storing the final episode reward, as well as the intra-instance results.

In [ ]:
def runBraitenbergAndStore(cur, con, batch_size: int, agent_dict: dict, yaml_files, task_names, temp_folder_location, agent_inference = False, port_base = 6600, randomise_port = True, verbose = True):
    
    # first, check if this agent has been added to the DB already

    agentid = selectID(cur, id_name = "agentid", table_name = "vbraitenbergvehicles", WHERE_column = "agent_tag", WHERE_clause = agent_dict['agent_tag'], secondary_WHERE_column = "aai_seed", secondary_WHERE_clause = agent_dict['aai_seed'])

    try:
        task_names, yaml_files = removePreviouslyRunInstances(cur = cur, yaml_files=yaml_files, task_names=task_names, agentid=agentid, agent_table = "vbraitenbergvehicles", agent_instance_results_table = "vbraitenbergvehicleinstanceresults")
    except:
        if verbose:
            print("Running on all files.")

    
    # now proceed with testing
    yaml_index = 0

    if randomise_port:

        port = port_base + yaml_index + random.randint( #create random base port.
            0, 9000
            )
        
    else:
        port = port_base + yaml_index


    batch_counter = 0

    #set seed
    random.seed(agent_dict['aai_seed'])

    if len(yaml_files) > 0:
        for yaml_index in range(0, len(yaml_files), batch_size):

            if (yaml_index + batch_size) > len(yaml_files) or batch_size > len(yaml_files):
                upper_bound = len(yaml_files)
            else:
                upper_bound = (yaml_index + batch_size)

            if verbose:
                print(f"Running inferences on batch {batch_counter + 1} of {batch_size} files of total {len(yaml_files)}. {len(yaml_files) - (batch_size * (batch_counter + 1))} instances to go.")

            batch_files = yaml_files[yaml_index:upper_bound]

            batch_file_names = task_names[yaml_index:upper_bound]

            batch_temp_file_name = f"TempConfig_{agent_dict['agent_tag']}_{agent_dict['aai_seed']}_{yaml_index}.yml"

            config_file_path = yaml_combinor(file_list = batch_files, temp_file_location=temp_folder_location, stored_file_name = batch_temp_file_name)

            agent = vanillaBraitenberg(agent_dict['no_rays'], agent_dict['angles']) # initialise agent class

            if verbose:
                print("Opening AAI Environment.")

            temp_port = port + yaml_index # increment through ports to prevent calling the same socket.

            aai_env = AnimalAIEnvironment( 
                inference=agent_inference, #Set true when watching the agent
                seed = agent_dict['aai_seed'],
                worker_id=temp_port,
                file_name=env_path,
                arenas_configurations=config_file_path,
                base_port=temp_port,
                useCamera=False,
                useRayCasts = True,
                raysPerSide=int((agent_dict['no_rays'])/2),
                rayMaxDegrees=agent_dict['angles'],
                no_graphics=(not agent_inference)
            )

            behavior = list(aai_env.behavior_specs.keys())[0] # by default should be AnimalAI?team=0

            firststep = True

            for _instance in range(len(batch_files)): 

                #get instance ID

                instanceid = selectID(cur, id_name = "instanceid", table_name = "instances", WHERE_column = "instancename", WHERE_clause = batch_file_names[_instance])

                #prepare to run instance

                if firststep:
                    aai_env.step() # take first step to get an observation
                    firststep = False
                    
                dec, term = aai_env.get_steps(behavior)

                done = False

                episodeReward = 0

                step_counter = 0
    
                while not done:
                        
                    observations = aai_env.get_obs_dict(dec.obs)

                    action = agent.get_action(observations)

                    aai_env.set_actions(behavior, action.action_tuple)

                    aai_env.step()

                    step_counter += 1

                    dec, term = aai_env.get_steps(behavior)

                    if len(dec.reward) > 0 and len(term) <= 0:
                        episodeReward += dec.reward
                        try:
                            intraInstanceQuery = f"INSERT INTO vbraitenbergvehicleintrainstanceresults(instanceid, agentid, step, actiontaken, stepreward, xvelocity, yvelocity, zvelocity, xpos, ypos, zpos) VALUES ({instanceid}, {agentid}, {step_counter}, '{action.name}', {float(episodeReward)}, {observations['velocity'][0]}, {observations['velocity'][1]}, {observations['velocity'][2]}, {observations['position'][0]}, {observations['position'][1]}, {observations['position'][2]});"
                            cur.execute(intraInstanceQuery)
                            con.commit()
                         
                        except:
                            print(f"There's something wrong with this step. Here's the query {intraInstanceQuery}")
                            pass

                    elif len(term) > 0: #Episode is over
                        episodeReward += term.reward
                        if verbose:
                            print(f"Episode Reward: {episodeReward}")
                        done = True
                        firststep = True

                        try:
                            intraInstanceQuery = f"INSERT INTO vbraitenbergvehicleintrainstanceresults(instanceid, agentid, step, actiontaken, stepreward, xvelocity, yvelocity, zvelocity, xpos, ypos, zpos) VALUES ({instanceid}, {agentid}, {step_counter}, '{action.name}', {float(episodeReward)}, {observations['velocity'][0]}, {observations['velocity'][1]}, {observations['velocity'][2]}, {observations['position'][0]}, {observations['position'][1]}, {observations['position'][2]});"
                            cur.execute(intraInstanceQuery)
                            #con.commit()
                            
                        except:
                            print(f"There's something wrong with this step. Here's the query {intraInstanceQuery}")
                            pass
                             
                        try:
                            insertInstanceResults = f"INSERT INTO vbraitenbergvehicleinstanceresults(instanceid, agentid, finalreward) VALUES ({instanceid}, {agentid}, {float(episodeReward)});"
                            cur.execute(insertInstanceResults)
                            #con.commit()
                            if verbose:
                                print("Pushing results to database.")
                        except:
                            print("It looks like this agent has already been tested on this instance.")
                    
                    else:
                        pass

            aai_env.close()
            if verbose:
                print("Moving to next batch.")
            batch_counter += 1
            con.commit()
            os.remove(config_file_path)
        else:
            if verbose:
                print("This agent has already been run on all of these tasks.")
    

In [ ]:
def run_agent_on_instance_wrapper(seed, agent, yaml_batch_size=1, port_base = 6600, randomise_port = True, verbose = True):
    agent['aai_seed'] = seed
    if verbose:
        print(f"Running {agent['agent_tag']} on seed {seed}.")
    
    mycursor, connection = databaseConnector('databaseConnectionDetails.csv')

    runBraitenbergAndStore(mycursor, connection, yaml_batch_size, agent_dict=agent, yaml_files=yaml_files, task_names=task_names, temp_folder_location=temp_folder_location, agent_inference=False, port_base = port_base, randomise_port = randomise_port, verbose = verbose)

    mycursor.close()

In [ ]:
yaml_batch_size = 1 #problem with task ordering, so having to do batches of 1. Much slower...
counter = 0
verbose = True

while counter <= (len(seeds_to_run) * len(agent_dict_list)):
    try:
        for seed in seeds_to_run:
            for agent_dictionary in agent_dict_list:
                adhoc_port = counter+10
                run_agent_on_instance_wrapper(seed, agent_dictionary, yaml_batch_size=yaml_batch_size, port_base = adhoc_port, randomise_port = True, verbose = verbose)
                if verbose:
                     print("Moving to next seed.")
                counter += 1
            if counter > (len(seeds_to_run) + len(agent_dict_list)):
                break
            if verbose:
                 print("Moving to next agent.")
    except:
        print("Sockets were occupied. Waiting 10 seconds and starting again.")
        counter = 0
        time.sleep(10)